In [37]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import joblib
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import numpy as np

In [38]:
# Caricamento del dataset
file_path = 'data-istruzioni/Data.xlsx'
data = pd.read_excel(file_path)

# Rimozione colonne inutili
data_cleaned = data.drop(columns=['ID'])

# Suddivisione in Training, Validation e Test set prima di gestire i valori nulli
X = data_cleaned.iloc[:, :-1]
y = data_cleaned.iloc[:, -1]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Gestione valori mancanti: sostituisce i NaN con la media per colonne numeriche
numerical_imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(numerical_imputer.fit_transform(X_train), columns=X_train.columns)
X_val = pd.DataFrame(numerical_imputer.transform(X_val), columns=X_val.columns)
X_test = pd.DataFrame(numerical_imputer.transform(X_test), columns=X_test.columns)

# Verifica dimensioni dei set
print(f"Training set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

Training set: (55, 249), Validation set: (12, 249), Test set: (12, 249)


In [39]:
# Selezione delle migliori 10 feature (solo sui dati di training)
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train, y_train)

# Trasforma anche il Validation e il Test set
X_val_selected = selector.transform(X_val)
X_test_selected = selector.transform(X_test)

# Salva il selettore per future predizioni
joblib.dump(selector, 'feature_selector.pkl')

/home/elly/miniconda3/envs/NYC/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:111: UserWarning: Features [222 223 224 234 246] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/elly/miniconda3/envs/NYC/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


['feature_selector.pkl']

In [40]:
# Addestramento del modello Random Forest
model = RandomForestClassifier(n_estimators=200, max_depth=100, class_weight='balanced', random_state=42)
model.fit(X_train_selected, y_train)

# Salva il modello
joblib.dump(model, 'final_model.pkl')

['final_model.pkl']

In [41]:
# Mostra le feature selezionate
selected_features = X.columns[selector.get_support()]
print("Feature selezionate:", selected_features.tolist())

Feature selezionate: ['Epoca_TC', 'Area_grasso_periviscerale', 'GS_alla_diagnosi', 'TNM_alla_diagnosi', 'Eta_alla_RP', 'GLCM3_entropy_-1_0_-1', 'GLCM3_inverse_1_-1_0', 'GLCM3_entropy_1_0_-1', 'GLCM3_autocorrelation_1_0_1', 'LBP_TOP_range']


In [42]:
# Valutazione sul Validation set
y_val_pred = model.predict(X_val_selected)

print("\nValutazione sul Validation Set:")
print(classification_report(y_val, y_val_pred))
print("\nMatrice di confusione sul Validation Set:")
print(confusion_matrix(y_val, y_val_pred))


Valutazione sul Validation Set:
              precision    recall  f1-score   support

           0       0.33      0.20      0.25         5
           1       0.56      0.71      0.62         7

    accuracy                           0.50        12
   macro avg       0.44      0.46      0.44        12
weighted avg       0.46      0.50      0.47        12


Matrice di confusione sul Validation Set:
[[1 4]
 [2 5]]


In [43]:
# Valutazione sul Test set
y_test_pred = model.predict(X_test_selected)

print("\nValutazione sul Test Set:")
print(classification_report(y_test, y_test_pred))
print("\nMatrice di confusione sul Test Set:")
print(confusion_matrix(y_test, y_test_pred))


Valutazione sul Test Set:
              precision    recall  f1-score   support

           0       0.40      0.40      0.40         5
           1       0.57      0.57      0.57         7

    accuracy                           0.50        12
   macro avg       0.49      0.49      0.49        12
weighted avg       0.50      0.50      0.50        12


Matrice di confusione sul Test Set:
[[2 3]
 [3 4]]


In [44]:
# Valutazione globale del modello
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred)
recall = recall_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

print("\nMetriche globali sul Test Set:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Metriche globali sul Test Set:
Accuracy: 0.50
Precision: 0.57
Recall: 0.57
F1-Score: 0.57


In [45]:
# Cross-Validation (solo sul Validation set)
cv_scores = cross_val_score(model, X_val_selected, y_val, cv=5, scoring='accuracy')

print("\nCross-Validation Scores (Accuracy):", cv_scores)
print(f"Mean Accuracy: {cv_scores.mean():.2f}")
print(f"Standard Deviation: {cv_scores.std():.2f}")


Cross-Validation Scores (Accuracy): [0.  0.  0.  1.  0.5]
Mean Accuracy: 0.30
Standard Deviation: 0.40


In [46]:
# Funzione per testare tutti i dati nel test set
def test_full_test_set():
    # Trasforma il test set con il selettore di feature
    X_test_selected = selector.transform(X_test)

    # Predizioni sul test set
    predictions = model.predict(X_test_selected)

    # Confronto con i valori reali
    correct_predictions = (y_test == predictions).sum()
    total_predictions = len(y_test)
    accuracy_on_test = correct_predictions / total_predictions

    print("\nRisultati sui dati di Test Set:")
    for i, pred in enumerate(predictions):
        print(f"Esempio {i+1}: {'Recidiva' if pred == 1 else 'Non Recidiva'} (Reale: {'Recidiva' if y_test.iloc[i] == 1 else 'Non Recidiva'})")

    print(f"\nNumero di predizioni corrette: {correct_predictions}")
    print(f"Totale predizioni: {total_predictions}")
    print(f"Accuracy sul Test Set: {accuracy_on_test:.2f}")


In [47]:
test_full_test_set()



Risultati sui dati di Test Set:
Esempio 1: Recidiva (Reale: Non Recidiva)
Esempio 2: Recidiva (Reale: Recidiva)
Esempio 3: Recidiva (Reale: Recidiva)
Esempio 4: Recidiva (Reale: Recidiva)
Esempio 5: Recidiva (Reale: Non Recidiva)
Esempio 6: Recidiva (Reale: Recidiva)
Esempio 7: Recidiva (Reale: Non Recidiva)
Esempio 8: Non Recidiva (Reale: Recidiva)
Esempio 9: Non Recidiva (Reale: Non Recidiva)
Esempio 10: Non Recidiva (Reale: Recidiva)
Esempio 11: Non Recidiva (Reale: Non Recidiva)
Esempio 12: Non Recidiva (Reale: Recidiva)

Numero di predizioni corrette: 6
Totale predizioni: 12
Accuracy sul Test Set: 0.50
